# Imports

In [2]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import plotly.express as plx
import matplotlib.pyplot as plt
from wordcloud import WordCloud , STOPWORDS
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

tqdm.pandas()

# Loading Dataset

## Downloading Dataset

In [3]:
!gdown 1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW
!gdown 1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB
!gdown 1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h

Downloading...
From: https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW
To: /content/train_data.csv
100% 635M/635M [00:02<00:00, 241MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB
To: /content/test_data.csv
100% 15.6M/15.6M [00:00<00:00, 139MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h
To: /content/title_brand.csv
100% 97.3M/97.3M [00:00<00:00, 254MB/s]


In [7]:
train_data = pd.read_csv('train_data.csv')
title_brand = pd.read_csv('title_brand.csv')

<ipython-input-7-3e25fee2e3c3>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv')


#Section 3

In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding , LSTM , Dense , SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.metrics import F1Score

texts = train_data['reviewText']
labels = train_data['overall']
del train_data


max_words = 1000
max_len = 200
tokenizer = Tokenizer(num_words = max_words , oov_token = '<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences , maxlen = max_len , padding='post', truncating='post')

labels_categorical = tf.keras.utils.to_categorical(labels)

X_train , X_validation , y_train , y_validation = train_test_split(padded_sequences , labels_categorical , test_size = 0.1 , random_state = 42)

model = Sequential()
model.add(Embedding(input_dim = max_words , output_dim = 128 , input_length= max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(6 , activation = 'softmax'))

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = [F1Score(average = 'micro')])

model.fit(X_train , y_train , epochs = 10 , batch_size = 32 , validation_data =(X_validation , y_validation))

Epoch 1/10
23596/23596 [==============================] - 313s 13ms/step - loss: 0.9427 - f1_score: 0.6394 - val_loss: 0.7768 - val_f1_score: 0.6933
Epoch 2/10
23596/23596 [==============================] - 286s 12ms/step - loss: 0.7570 - f1_score: 0.6991 - val_loss: 0.7291 - val_f1_score: 0.7112
Epoch 3/10
23596/23596 [==============================] - 283s 12ms/step - loss: 0.7203 - f1_score: 0.7132 - val_loss: 0.7129 - val_f1_score: 0.7185
Epoch 4/10
23596/23596 [==============================] - 280s 12ms/step - loss: 0.7017 - f1_score: 0.7200 - val_loss: 0.7025 - val_f1_score: 0.7213
Epoch 5/10
 7431/23596 [========>.....................] - ETA: 2:55 - loss: 0.6866 - f1_score: 0.7250

KeyboardInterrupt: ignored